In [12]:
import pandas as pd
import glob
import os

In [13]:
# Definir o caminho da pasta onde os arquivos CSV estão localizados
caminho = os.path.join("files", "energy_ microinversores")
print(caminho)

files/energy_ microinversores


In [14]:
# Função para obter todos os arquivos CSV na pasta
def obter_arquivos_csv(caminho):
    """Retorna uma lista de caminhos dos arquivos CSV no diretório especificado."""
    arquivos = glob.glob(os.path.join(caminho, '*.csv'))
    if not arquivos:
        print("Nenhum arquivo CSV encontrado.")
    return arquivos

In [15]:
# Função para ler e processar os arquivos CSV
def ler_arquivos_csv(arquivos_csv):
    """Lê todos os arquivos CSV e retorna uma lista de DataFrames."""
    lista = []
    for arquivo in arquivos_csv:
        try:
            df = pd.read_csv(arquivo, index_col=None, header=0, low_memory=False, parse_dates=['Date'])
            lista.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    return lista

In [16]:
# Função para concatenar os DataFrames
def concatenar_dataframes(lista):
    """Concatena todos os DataFrames em um único DataFrame."""
    if not lista:
        print('Nenhum dado para concatenar.')
        return None
    return pd.concat(lista, ignore_index=True)

In [17]:
# Função para criar uma nova coluna contendo o nome do microinversor baseado no SN
def adicionar_id_e_nome_microinversor(df):
    """
    Adiciona uma coluna 'ID' no início do DataFrame e uma coluna 'Microinversor' baseada na coluna 'SN'.
    Associa cada SN a um nome de microinversor único.
    """
    # Adicionar a coluna 'ID' com valores sequenciais
    df.insert(0, 'ID', range(1, len(df) + 1))

    if 'SN' not in df.columns and 'Sn' in df.columns:
        df.rename(columns={'Sn': 'SN'}, inplace=True)

    if 'SN' not in df.columns:
        print("Erro: A coluna 'SN' não foi encontrada no arquivo.")
        return df

    # Criar um dicionário associando os SNs a nomes de microinversores
    microinversores_dict = {sn: f"Microinversor_{idx+1}" for idx, sn in enumerate(df['SN'].unique())}

    # Mapear a coluna SN para os nomes dos microinversores
    df['Microinversor'] = df['SN'].map(microinversores_dict)

    return df

In [19]:
# Função para salvar o DataFrame atualizado
def salvar_dataframe(df_final, caminho_novo_arquivo):
    """Salva o DataFrame atualizado em um arquivo CSV."""
    try:
        df_final.to_csv(caminho_novo_arquivo, index=False)
        print(f"Arquivo salvo com sucesso: {caminho_novo_arquivo}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

# Função principal
def main():
    caminho_novo_arquivo = os.path.join(caminho, 'arquivo.csv')

    arquivos_csv = obter_arquivos_csv(caminho)
    if not arquivos_csv:
        return  # Encerra o script se não houver arquivos

    lista = ler_arquivos_csv(arquivos_csv)
    df_novo = concatenar_dataframes(lista)

    if df_novo is not None:
        df_novo = adicionar_id_e_nome_microinversor(df_novo)
        display(df_novo.head())
        salvar_dataframe(df_novo, caminho_novo_arquivo)

In [20]:
if __name__ == "__main__":
    main()

,ID,Plant Name,Date,SN,Port,Energy (kWh),Microinversor
0,1,Wilkne,2024-11-27,106272403152,1,2.131,Microinversor_1
1,2,Wilkne,2024-11-27,106272403152,2,2.109,Microinversor_1
2,3,Wilkne,2024-11-27,106272403152,3,2.105,Microinversor_1
3,4,Wilkne,2024-11-27,106272403152,4,2.152,Microinversor_1
4,5,Wilkne,2024-11-27,106272403916,1,2.098,Microinversor_2


Arquivo salvo com sucesso: files/energy_ microinversores/arquivo.csv


In [21]:
#TODO: Função temporaria
df_temp = pd.read_csv(os.path.join(caminho, 'arquivo.csv'))
df_temp.head()

,ID,Plant Name,Date,SN,Port,Energy (kWh),Microinversor
0,1,Wilkne,2024-11-27,106272403152,1,2.131,Microinversor_1
1,2,Wilkne,2024-11-27,106272403152,2,2.109,Microinversor_1
2,3,Wilkne,2024-11-27,106272403152,3,2.105,Microinversor_1
3,4,Wilkne,2024-11-27,106272403152,4,2.152,Microinversor_1
4,5,Wilkne,2024-11-27,106272403916,1,2.098,Microinversor_2


In [22]:
valores_unicos = df_temp['SN'].unique()

print(valores_unicos)

[106272403152 106272403916 106272404964 116191966583]


In [23]:
# Substituir os valores na coluna 'SN'
df_temp['SN'] = df_temp['SN'].replace(106272404964, 116191966583)

# Verificar se a substituição foi feita corretamente
print(df_temp['SN'].unique())

[106272403152 106272403916 116191966583]


In [24]:
# Contar a frequência dos valores na coluna 'SN'
frequencia_valores = df_temp['SN'].value_counts()

print(frequencia_valores)

SN
106272403152    4816
106272403916    4816
116191966583    4816
Name: count, dtype: int64
